# Data Augmentation

In [1]:
from keras.preprocessing.image import ImageDataGenerator

# Create image augmenter
dataAugment = ImageDataGenerator(rescale=1.0/255,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 shear_range=0.2,
                                 fill_mode='reflect',
                                 horizontal_flip=True,
                                 validation_split=0.2,
                                 data_format='channels_last'
                                )
        
trainDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                     target_size=(224, 224), # change this to the input size used for your model
                                                     class_mode='categorical',
                                                     subset='training',
                                                     shuffle=True
                                                     # batch_size=
                                                    )

validationDataGenerator = dataAugment.flow_from_directory(directory='Data/Train',
                                                          target_size=(224,224), # change this to the input size used for your model
                                                          class_mode='categorical',
                                                          subset='validation',
                                                          shuffle=True
                                                          # batch_size=
                                                         )

Using TensorFlow backend.


Found 221 images belonging to 3 classes.
Found 54 images belonging to 3 classes.


# Xception Model

In [2]:
from keras.applications.xception import Xception

xception_base = Xception(input_shape=(224,224,3), include_top=False, weights='imagenet')
xception_base.summary()

83689472/83683744 [==============================] - 16s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________

In [7]:
# Freeze all layers
for layer in xception_base.layers:
    layer.trainable = False
# Unfreeze last 6 layers
for layer in xception_base.layers[-6:]:
    layer.trainable = True

# Check the trainable status of the individual layers
for layer in xception_base.layers:
    if layer.trainable == True:
        print(layer, layer.trainable)

<keras.layers.convolutional.SeparableConv2D object at 0xb3b043048> True
<keras.layers.normalization.BatchNormalization object at 0xb3b070dd8> True
<keras.layers.core.Activation object at 0xb3b070eb8> True
<keras.layers.convolutional.SeparableConv2D object at 0xb3b0aab00> True
<keras.layers.normalization.BatchNormalization object at 0xb3b0eb4a8> True
<keras.layers.core.Activation object at 0xb3b184e10> True


In [8]:
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers, models

model = models.Sequential()
model.add(xception_base)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

sgd_opt = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=True)
Nadam_opt = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999,
                             epsilon=None, schedule_decay=0.004) # recommended to leave the default params

model.compile(loss='categorical_crossentropy',
              optimizer=Nadam_opt,
              metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25690368  
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
Total params: 46,552,619
Trainable params: 30,439,939
Non-trainable params: 16,112,680
_________________________________________________________________


In [ ]:
model.fit_generator(generator=trainDataGenerator,
                    steps_per_epoch=trainDataGenerator.samples/trainDataGenerator.batch_size,
                    epochs=10,
                    validation_data=validationDataGenerator,
                    validation_steps=validationDataGenerator.samples/validationDataGenerator.batch_size,
                    verbose=1
                   )

In [ ]:
# Possible hyperparameters:
# Num. of layers to freeze
# Num. of hidden layers in fc layer
# Num. of neurons per hidden layer
# Optimizer learning rate
# Dropout percentage